In [ ]:
pip install boto3

In [3]:
import os
import random
import shutil
from glob import glob
from tqdm import tqdm

# --- Cấu hình Đường dẫn và Tỷ lệ ---
# Thư mục GỐC chứa 'images' và 'labels'
root_dir = r'D:\Document_Layout\project-17' 

# Thư mục đầu ra, nơi sẽ tạo các thư mục 'images' và 'labels' mới (train/val)
output_dir = r'D:\Document_Layout\data'

train_ratio = 0.8 # 80%
val_ratio = 0.2   # 20%
SEED = 42         # Dùng seed để kết quả chia luôn giống nhau

# Định nghĩa thư mục chứa ảnh và nhãn HIỆN TẠI
images_dir = os.path.join(root_dir, 'images')
labels_dir = os.path.join(root_dir, 'labels')

# --- 1. Tạo các thư mục đích ---
print(f"Kiểm tra và tạo thư mục đích tại: {output_dir}")
os.makedirs(os.path.join(output_dir, 'images', 'train'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'labels', 'train'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'images', 'val'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'labels', 'val'), exist_ok=True)

# --- 2. Lấy danh sách tất cả các file ảnh và đuôi tương ứng ---
image_extensions = ['*.jpg', '*.jpeg', '*.png']
image_files = []
# QUAN TRỌNG: Quét file ảnh trong thư mục 'images_dir'
for ext in image_extensions:
    image_files.extend(glob(os.path.join(images_dir, ext)))

# Tạo danh sách các tuple (tên_gốc, đuôi_file)
base_names_with_ext = []
for f in image_files:
    # Lấy tên file gốc (ví dụ: '0b7c2abc-1...') và đuôi (.png)
    name, ext = os.path.splitext(os.path.basename(f))
    base_names_with_ext.append((name, ext))

# --- 3. Trộn ngẫu nhiên và Chia danh sách ---
random.seed(SEED) 
random.shuffle(base_names_with_ext)

num_images = len(base_names_with_ext)

if num_images == 0:
    print("-" * 50)
    print(f"LỖI: Không tìm thấy file ảnh nào trong thư mục: {images_dir}")
    print("Vui lòng kiểm tra lại đường dẫn và định dạng file ảnh.")
    print("-" * 50)
    exit() 

num_train = int(num_images * train_ratio)

train_names_with_ext = base_names_with_ext[:num_train]
val_names_with_ext = base_names_with_ext[num_train:]

print(f"Tổng số ảnh tìm thấy: {num_images}")
print(f"Ảnh Train (80%): {len(train_names_with_ext)}")
print(f"Ảnh Val (20%): {len(val_names_with_ext)}")

# --- 4. Hàm sao chép file (ảnh và nhãn) ---
def move_files(file_list_with_ext, subset_name):
    print(f"\n--- Sao chép {subset_name} ---")
    for name, ext in tqdm(file_list_with_ext): 
        # 1. Sao chép Ảnh (Lấy từ thư mục images_dir)
        src_img = os.path.join(images_dir, f'{name}{ext}') 
        dst_img = os.path.join(output_dir, 'images', subset_name, f'{name}{ext}')
        
        # 2. Sao chép Nhãn (Lấy từ thư mục labels_dir)
        src_label = os.path.join(labels_dir, f'{name}.txt')
        dst_label = os.path.join(output_dir, 'labels', subset_name, f'{name}.txt')
        
        # Sao chép Ảnh
        if os.path.exists(src_img):
            shutil.copy(src_img, dst_img)

        # Sao chép Nhãn
        if os.path.exists(src_label):
            shutil.copy(src_label, dst_label)
        else:
            print(f"CẢNH BÁO: Không tìm thấy nhãn tương ứng cho ảnh: {name}.txt.")


# --- 5. Thực hiện sao chép ---
move_files(train_names_with_ext, 'train')
move_files(val_names_with_ext, 'val')

print("\nQuá trình chia dữ liệu hoàn tất!")

Kiểm tra và tạo thư mục đích tại: D:\Document_Layout\data
Tổng số ảnh tìm thấy: 221
Ảnh Train (80%): 176
Ảnh Val (20%): 45

--- Sao chép train ---


100%|██████████| 176/176 [00:00<00:00, 472.87it/s]



--- Sao chép val ---


100%|██████████| 45/45 [00:00<00:00, 483.62it/s]


Quá trình chia dữ liệu hoàn tất!


In [14]:
import mlflow
import os

os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://34.72.241.173:9000"
os.environ["MLFLOW_EXPERIMENT_NAME"] = "Document_Layout_Experiment"
os.environ["MLFLOW_RUN"] = "Document_Layout_Experiment_Run_backbone" 

mlflow.set_tracking_uri("http://34.72.241.173:5000")
mlflow.enable_system_metrics_logging()

In [15]:
from ultralytics import YOLO

# --- Cấu hình Đường dẫn ---
# Đường dẫn đến file YAML đã tạo
data_yaml_path = r'D:\Document_Layout\data\data.yaml' 

# --- 1. Tải Mô hình Cơ sở YOLOv8 ---
# Sử dụng 'yolov8n.pt' (nano) để bắt đầu quá trình Transfer Learning.
# YOLO sẽ tự động tải file này nếu nó chưa có.
model = YOLO('yolov8n-doclaynet.pt') 

# --- 2. Huấn luyện Mô hình ---
print(f"Bắt đầu huấn luyện YOLOv8n với dữ liệu mới: {data_yaml_path}")
print(f"Tổng số epochs: 100")

# Lệnh train:
# data: File cấu hình dữ liệu
# epochs: Số lượng epochs muốn train
# imgsz: Kích thước đầu vào của ảnh
# name: Tên thư mục kết quả sẽ lưu trong 'runs/detect/'
results = model.train(
    data=data_yaml_path,
    epochs=20,
    imgsz=640,
    batch=8,
    name='document_layout_v3_new_start',
    # Thêm các tham số learning rate ở đây
    lr0=0.01,  # Tốc độ học ban đầu (giá trị mặc định trong YOLOv8)
    lrf=0.001,  # Tốc độ học cuối cùng sau khi điều chỉnh theo lịch trình
    momentum=0.937,
    optimizer='SGD',
    # --- Tham số ĐÓNG BĂNG ---
    freeze=['backbone']

)
print("\nQuá trình huấn luyện đã hoàn tất!")
print(f"Kết qu ++ả (bao gồm file best.pt mới) được lưu tại: runs/detect/document_layout_v3_new_start/")

Bắt đầu huấn luyện YOLOv8n với dữ liệu mới: D:\Document_Layout\data\data.yaml
Tổng số epochs: 100
New https://pypi.org/project/ultralytics/8.3.233 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.231  Python-3.10.11 torch-2.9.1+cpu CPU (AMD Ryzen 5 5625U with Radeon Graphics)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:\Document_Layout\data\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=['backbone'], half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.001, mask_ratio=4, max_det=300, mixup=

2025/11/28 15:55:42 INFO mlflow.bedrock: Enabled auto-tracing for Bedrock. Note that MLflow can only trace boto3 service clients that are created after this call. If you have already created one, please recreate the client by calling `boto3.client`.
2025/11/28 15:55:42 INFO mlflow.tracking.fluent: Autologging successfully enabled for boto3.
2025/11/28 15:55:42 INFO mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics. Set logger level to DEBUG for more details.
2025/11/28 15:55:42 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


MLflow: logging run_id(78223f7036034715924c64a30be3ea8d) to http://34.72.241.173:5000
MLflow: disable with 'yolo settings mlflow=False'
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to D:\Document_Layout\runs\detect\document_layout_v3_new_start8
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       1/20         0G      1.313      3.394      1.213         76        640: 100% ━━━━━━━━━━━━ 22/22 2.9s/it 1:032.8sss
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 3/3 2.6s/it 7.7s5.4ss
                   all         45        350      0.694      0.133      0.124      0.102

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       2/20         0G      1.264      2.743      1.177         79        640: 100% ━━━━━━━━━━━━ 22/22 2.8s/it 1:022.9sss
                 Class     Images  Instances      Box(P          R    

2025/11/28 16:20:17 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/11/28 16:20:18 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


MLflow: results logged to http://34.72.241.173:5000
MLflow: disable with 'yolo settings mlflow=False'

Quá trình huấn luyện đã hoàn tất!
Kết qu ++ả (bao gồm file best.pt mới) được lưu tại: runs/detect/document_layout_v3_new_start/


In [1]:
from ultralytics import YOLO

# 1. Tải mô hình
model = YOLO('yolov8n-doclaynet.pt')

# 2. In thông tin chi tiết của mô hình
# Tham số verbose=True đảm bảo in ra danh sách từng lớp
print("--- Cấu trúc mô hình chi tiết (Backbone, Neck, Head) ---")
model.info(verbose=True)

--- Cấu trúc mô hình chi tiết (Backbone, Neck, Head) ---
Model summary: 129 layers, 3,012,993 parameters, 0 gradients, 8.2 GFLOPs


(129, 3012993, 0, 8.2049024)

In [2]:
from ultralytics import YOLO
import torch.nn as nn

# Tải mô hình
model = YOLO('yolov8n-doclaynet.pt')

# Lấy danh sách các lớp (module list)
model_layers = model.model.model

print("--- Danh sách các Module và Index ---")
for i, layer in enumerate(model_layers):
    # In index và tên lớp (loại module)
    print(f"Index {i:2d}: {layer.__class__.__name__}")

# YOLOv8 thường phân chia rõ ràng các module trong file .yaml:
# - indices[0-9]: Backbone
# - indices[10-17]: Neck (Nối các feature map)
# - index[18-21]: Head (Detect layer)

--- Danh sách các Module và Index ---
Index  0: Conv
Index  1: Conv
Index  2: C2f
Index  3: Conv
Index  4: C2f
Index  5: Conv
Index  6: C2f
Index  7: Conv
Index  8: C2f
Index  9: SPPF
Index 10: Upsample
Index 11: Concat
Index 12: C2f
Index 13: Upsample
Index 14: Concat
Index 15: C2f
Index 16: Conv
Index 17: Concat
Index 18: C2f
Index 19: Conv
Index 20: Concat
Index 21: C2f
Index 22: Detect
